In [1]:
%%capture
!pip install FinMind==1.4.8

In [ ]:
from loguru import logger
from pyecharts.charts import Page

import FinMind
from FinMind import plotting
from FinMind.data import DataLoader
from IPython.display import HTML, display

In [ ]:
def kline(
    data_loader: DataLoader, stock_id: str, start_date: str, end_date: str
):
    stock_data = data_loader.taiwan_stock_daily(stock_id, start_date, end_date)
    stock_data = data_loader.feature.add_kline_institutional_investors(
        stock_data
    )
    stock_data = data_loader.feature.add_kline_margin_purchase_short_sale(
        stock_data
    )
    # 繪製k線圖
    kline_plot = plotting.kline(stock_data)
    return kline_plot


def bar(data_loader: DataLoader, stock_id: str, start_date: str, end_date: str):
    df = data_loader.taiwan_stock_month_revenue(
        stock_id=stock_id, start_date=start_date, end_date=end_date
    )
    df["labels"] = (
        df[["revenue_year", "revenue_month"]]
        .astype(str)
        .apply(lambda date: f"{date[0]}-{date[1]}M", axis=1)
    )
    df["series"] = df["revenue"].map(lambda value: round(value * 1e-8, 2))
    bar_plot = plotting.bar(
        labels=df["labels"],
        series=df["series"],
        title="月營收",
        yaxis_color="orange",
        y_axis_name="億",
    )
    return bar_plot


def line(
    data_loader: DataLoader, stock_id: str, start_date: str, end_date: str
):
    df = data_loader.taiwan_stock_shareholding(
        stock_id=stock_id, start_date=start_date, end_date=end_date
    )
    df["series"] = df["ForeignInvestmentSharesRatio"].map(
        lambda value: round(value * 1e-2, 2)
    )
    df["labels"] = df["date"]
    line_plot = plotting.line(
        labels=df["labels"],
        series=df["series"],
        title="外資持股比例",
        yaxis_color="blue",
        y_axis_name="",
    )
    return line_plot


def pie(data_loader: DataLoader, stock_id: str, start_date: str, end_date: str):
    df = data_loader.taiwan_stock_holding_shares_per(
        stock_id=stock_id, start_date=start_date, end_date=end_date
    )
    df = df[df["date"] == max(df["date"])]
    df = df[df["HoldingSharesLevel"] != "total"]
    df["labels"] = df["HoldingSharesLevel"]
    df["series"] = df["percent"]
    pie_plot = plotting.pie(
        labels=df["labels"], series=df["series"], title="股權分散表"
    )
    return pie_plot

In [ ]:
def dashboard(stock_id, start_date, end_date):
  data_loader = DataLoader()
  page = Page(layout=Page.SimplePageLayout)
  page.add(
      kline(data_loader, stock_id, start_date, end_date),
      bar(data_loader, stock_id, start_date, end_date),
      line(data_loader, stock_id, start_date, end_date),
      pie(data_loader, stock_id, start_date, end_date)
  )
  page.render("dashboard.html")
  display(HTML(filename="dashboard.html"))

In [ ]:
dashboard(stock_id='2330', start_date='2021-01-01', end_date='2021-08-01')